In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

class InvoicePublisher:

    def __init__(self):
        self.landing_zone = "/Volumes/dev/retail_store/landing_zone"
        self.bootstrap_server = 'pkc-619z3.us-east1.gcp.confluent.cloud:9092'
        self.jaas_module = 'org.apache.kafka.common.security.plain.PlainLoginModule'
        self.api_key = 'OVPNJYWE56QVX4LD'
        self.api_secret = 'LSK7hPY5Kvqpj12hiaZuLyawzvN+eOXqDXtOgyRdBAcz9j6RxQdx0DlIlSXLU7Jb'
        self.topic = 'invoices'
        self.delivery_add_schema = (StructType([
            StructField("AddressLine", StringType(), True),
            StructField("City", StringType(), True),
            StructField("State", StringType(), True),
            StructField("PinCode", StringType(), True),
            StructField("ContactNumber", StringType(), True)
        ]))
        self.line_item_schema = (StructType([
            StructField("ItemCode", StringType(), True),
            StructField("ItemDescription", StringType(), True),
            StructField("ItemPrice", DoubleType(), True),
            StructField("ItemQty", IntegerType(), True),
            StructField("TotalValue", DoubleType(), True)
        ]))
        self.invoice_schema = (StructType([
            StructField("InvoiceNumber", StringType(), True),
            StructField("CreatedTime", LongType(), True),
            StructField("StoreID", StringType(), True),
            StructField("PosID", StringType(), True),
            StructField("CashierID", StringType(), True),
            StructField("CustomerType", StringType(), True),
            StructField("CustomerCardNo", IntegerType(), True),
            StructField("TotalAmount", DoubleType(), True),
            StructField("NumberOfItems", IntegerType(), True),
            StructField("PaymentMethod", StringType(), True),
            StructField("TaxableAmount", DoubleType(), True),
            StructField("CGST", DoubleType(), True),
            StructField("SGST", DoubleType(), True),
            StructField("CESS", DoubleType(), True),
            StructField("DeliveryType", StringType(), True),
            StructField("DeliveryAddress", self.delivery_add_schema),
            StructField("InvoiceLineItems", ArrayType(self.line_item_schema))]))
    
    def load_raw_data(self):
        raw_df = (spark
                  .readStream
                  .format("json")
                  .schema(self.invoice_schema)
                  .load(self.landing_zone)
                  )
        return raw_df
    
    def format_message(self, raw_df):
        formatted_df = (raw_df.selectExpr("StoreID as key","to_json(struct(*)) as value"))
        return formatted_df
    
    def publish_to_kafka(self, formatted_df):
        write_streaming_query = (formatted_df
                                 .writeStream
                                 .format("kafka")
                                 .option("kafka.bootstrap.servers", self.bootstrap_server)
                                 .option("kafka.security.protocol", "SASL_SSL") 
                                 .option("kafka.sasl.mechanism", "PLAIN")
                                 .option("kafka.sasl.jaas.config", f"{self.jaas_module} required username='{self.api_key}' password='{self.api_secret}';")
                                 .option("topic", self.topic)
                                 .outputMode("append")
                                 .option("checkpointLocation", "/Volumes/dev/retail_store/checkpoint_invoices_raw")
                                 .start()
                                 )
        return write_streaming_query
    
    def start_publisher(self):
        raw_df = self.load_raw_data()
        formatted_df = self.format_message(raw_df)
        return self.publish_to_kafka(formatted_df)

In [0]:
publisher = InvoicePublisher()
write_streaming_query= publisher.start_publisher()

In [0]:
write_streaming_query.close()

---------------------------------------------------------------------------
AttributeError                            Traceback (most recent call last)
File <command-2848266856789996>, line 1
----> 1 write_streaming_query.close()

AttributeError: 'StreamingQuery' object has no attribute 'close'